# OpenAI API Connection Debug Notebook

This notebook helps debug OpenAI API connection issues and test the LLM service independently.

## Problem: "mismatched_project" error

The error means the `OPENAI_API_PROJECT` in your `.env` file doesn't match your API key.

## Solutions:
1. **Remove project ID** (recommended) - Comment out `OPENAI_API_PROJECT` in `.env`
2. **Get correct project ID** from https://platform.openai.com/settings
3. **Don't use project ID** - It's optional for most users


## 1. Load Environment Variables


In [ ]:
import os
from dotenv import load_dotenv

# Load .env file
load_dotenv()

# Check what's loaded
api_key = os.getenv('OPENAI_API_KEY')
model = os.getenv('OPENAI_DEFAULT_MODEL', 'gpt-5-nano')

print("Environment Variables:")
# print(f"  API Key: {'*' * 8}{api_key[-4:] if api_key else 'NOT SET'}")
print(f"  Default Model: {model}")


Environment Variables:
  API Key: ********cNwA
  Default Model: gpt-5-nano


## 2. Test OpenAI WITHOUT Project ID (This should work!)


In [19]:
from openai import OpenAI

# Test WITHOUT project ID
client = OpenAI(api_key=api_key, timeout=30)

try:
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": "Say 'Hello!' and nothing else."}]
    )
    print("✅ SUCCESS - OpenAI works WITHOUT project ID!")
    print(f"Response: {response.choices[0].message.content}")
    print(f"Model used: {response.model}")
    print(f"Tokens: {response.usage.total_tokens}")
    print("\n💡 Solution: Remove OPENAI_API_PROJECT from your .env file")
except Exception as e:
    print(f"❌ ERROR: {str(e)}")


✅ SUCCESS - OpenAI works WITHOUT project ID!
Response: Hello!
Model used: gpt-5-nano-2025-08-07
Tokens: 153

💡 Solution: Remove OPENAI_API_PROJECT from your .env file


## 3. Test Medical Note Summarization


In [20]:
sample_note = """Subjective: 45yo male presents with chest pain for 2 hours. 
Pain is substernal, 7/10 severity, radiating to left arm. 
Denies shortness of breath. Has history of hypertension.

Objective: BP 145/92, HR 88, RR 16, O2 sat 98% on RA. 
Appears uncomfortable but not in acute distress. 
Heart: regular rate and rhythm, no murmurs. Lungs clear.

Assessment: Chest pain, concerning for ACS.

Plan: EKG, troponin, cardiology consult, aspirin 325mg given."""

try:
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are a medical assistant. Summarize medical notes concisely."},
            {"role": "user", "content": f"Summarize this note:\n\n{sample_note}"}
        ],
        temperature=1.0
    )
    
    print("✅ SUMMARIZATION SUCCESS")
    print("="*60)
    print(response.choices[0].message.content)
    print("="*60)
    print(f"\nTokens used: {response.usage.total_tokens}")
except Exception as e:
    print(f"❌ ERROR: {str(e)}")


✅ SUMMARIZATION SUCCESS
Summary:

- 45-year-old man with 2 hours of substernal chest pain radiating to the left arm; denies SOB; hx hypertension.
- Vitals: BP 145/92, HR 88, RR 16, O2 98% on room air.
- Exam: uncomfortable but not in acute distress; regular rhythm, no murmurs; lungs clear.
- Assessment: Chest pain concerning for acute coronary syndrome (ACS).
- Plan: EKG, troponin, cardiology consult; aspirin 325 mg administered.

Tokens used: 596


## 4. Test Our LLM Service (after fixing .env)


In [16]:
# After you remove OPENAI_API_PROJECT from .env, test our service:
from app.services.llm import LLMService

try:
    llm_service = LLMService()
    result = llm_service.summarize_note(sample_note)
    
    print("✅ LLM SERVICE WORKS!")
    print("="*60)
    print(result['summary'])
    print("="*60)
    print(f"\nModel: {result['model_used']}")
    print(f"Tokens: {result['token_usage']}")
    print(f"Time: {result['processing_time_ms']}ms")
except Exception as e:
    print(f"❌ ERROR: {str(e)}")
    print("\nIf you see 'mismatched_project', remove OPENAI_API_PROJECT from .env")


⚠️  Removing OPENAI_API_PROJECT from environment
✓ Environment cleaned, ready to import service


OpenAI API error: Error code: 401 - {'error': {'message': 'OpenAI-Project header should match project for API key', 'type': 'invalid_request_error', 'code': 'mismatched_project', 'param': None}}


❌ ERROR: OpenAI API error: Error code: 401 - {'error': {'message': 'OpenAI-Project header should match project for API key', 'type': 'invalid_request_error', 'code': 'mismatched_project', 'param': None}}


Traceback (most recent call last):
  File "/Users/ryankennedy/Documents/freelance/df-healthbench/backend/app/services/llm.py", line 132, in _create_completion
    response = self.client.chat.completions.create(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/ryankennedy/Library/Caches/pypoetry/virtualenvs/df-backend--2HB9y44-py3.12/lib/python3.12/site-packages/openai/_utils/_utils.py", line 286, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/Users/ryankennedy/Library/Caches/pypoetry/virtualenvs/df-backend--2HB9y44-py3.12/lib/python3.12/site-packages/openai/resources/chat/completions/completions.py", line 1189, in create
    return self._post(
           ^^^^^^^^^^^
  File "/Users/ryankennedy/Library/Caches/pypoetry/virtualenvs/df-backend--2HB9y44-py3.12/lib/python3.12/site-packages/openai/_base_client.py", line 1259, in post
    return cast(ResponseT, self.request(cast_to, opts, stream=stream, stream_cls=stream_cls))
           